# Introduction to Azure OpenAI Function Calling


## Objective

This notebook demonstrates the following:

1. Generative AI
1. Function calling


## Time

You should expect to spend 2 minutes running this sample.


## About this example

The objective of the provided Python file is to learn about Azure OpenAI Function Calling using the Azure OpenAI API.


## Installation

Refer to the README.md file in this folder for installation instructions.


## Parameters


In [ ]:
import os
import json
import sqlite3
import pandas as pd
from openai import AzureOpenAI
from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Any, Callable, Dict

load_dotenv(".env")

api_endpoint = os.getenv("OPENAI_URI")
api_key = os.getenv("OPENAI_KEY")
api_version = os.getenv("OPENAI_VERSION")
api_deployment_name = os.getenv("OPENAI_GPT_DEPLOYMENT")

con = sqlite3.connect("./database/contoso-sales.db")

system_message = None

In [ ]:
def log_message(message: str):
    display(HTML(f'<span style="color: red;"><strong>{message}</strong></span>'))

In [ ]:
def display_wrapped(message: str):
    wrapped_content = f'<pre style="white-space: pre-wrap; word-wrap: break-word;">{message}</pre>'
    display(HTML(wrapped_content))

In [ ]:
def get_revenue_by_region(region: str = None) -> pd.DataFrame:
    params = []
    query = """
        SELECT  
            region AS Region,  
            SUM(number_of_orders) AS Orders,  
            SUM(revenue) AS Revenue,  
            SUM(discount) AS Discount,  
            SUM(shipping_cost) AS ShippingCost,
            SUM(revenue) - SUM(discount) - SUM(shipping_cost) AS NetRevenue  
        FROM 
            sales_data
    """

    if region:
        query += "WHERE UPPER(Region) = UPPER(?)\n"
        params.append(region)

    query += "GROUP BY Region\n"
    query += "ORDER BY NetRevenue DESC"

    return pd.read_sql_query(query, con, params=params)

In [ ]:
def get_sales_by_month(month: int, year: int) -> pd.DataFrame:
    query = """
        SELECT   
            year || '-' || CASE 
                WHEN month < 10 THEN '0' || month 
                ELSE month 
                END AS Month,
        SUM(revenue) AS Revenue,
        SUM(revenue) - SUM(discount) - SUM(shipping_cost) AS NetRevenue
        FROM sales_data 
        WHERE Month = ?  AND Year = ?
        GROUP BY Month, Year 
        ORDER BY Year, Month
    """

    return pd.read_sql_query(query, con, params=[month, year])

### Create an AzureOpenAI client


In [ ]:
client = AzureOpenAI(api_key=api_key, api_version=api_version, azure_endpoint=api_endpoint)

### Define the Assistant tools

The tools list defines the tools that the OpenAI Chat Completion will look for when generating a response. It's then up to the application to decide how to use the response. In this example corresponding Python functions are defined for each tool and the response is passed to the appropriate function.

In [ ]:
tools_list = [
    {
        "type": "function",
        "function": {
            "name": "get_revenue_by_region",
            "description": "Get the sales revenue for Contoso by region.",
            "parameters": {
                "type": "object",
                "properties": {
                    "region": {"type": "string"},
                },
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_sales_by_month",
            "description": "Get the Contoso sales by month.",
            "parameters": {
                "type": "object",
                "properties": {
                    "month": {
                        "type": "integer",
                        "description": "The Contoso sales for month.",
                    },
                    "year": {
                        "type": "integer",
                        "description": "The Contoso sales for year.",
                    },
                },
                "required": ["month", "year"],
            },
        },
    },
]

### Process Function calling

Responsible for calling the appropriate function based on the tool detected in the response. The function is called with the response and the tool as arguments.


In [ ]:
def call_functions(tool_calls) -> None:
    function_map: Dict[str, Callable[[Any], pd.DataFrame]] = {
        "get_revenue_by_region": lambda args: get_revenue_by_region(args.get("region", None)),
        "get_sales_by_month": lambda args: get_sales_by_month(args["month"], args["year"]),
    }

    for tool_call in tool_calls:
        func_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)

        log_message(f"Function Name: {func_name}, Function Args: {arguments}")

        function = function_map.get(func_name)
        if not function:
            raise ValueError(f"Unknown function: {func_name}")

        result_df = function(arguments)
        display(result_df)

## Instructions for role system assignment

Defines the system message for the role system assignment.



In [ ]:
system_message = (
    "You are a sales analysis assistant for Contoso. "
    "Please be polite, professional, helpful, and friendly. "
    "You get all the sales data from this app using the functions provided. "
    "If a question is not related to sales or you cannot answer the question, "
    "say, 'contact IT for more assistance.' "
    "If the user asks for help or says 'help', provide a list of sample questions that you can answer."
)

## Construct the Assistant and send to Azure OpenAI


In [ ]:
def process_message(question: str):

    messages = [{"role": "system", "content": system_message}, {"role": "user", "content": question}]

    response = client.chat.completions.create(
        model=api_deployment_name,
        messages=messages,
        tools=tools_list,
    )

    response_message = response.choices[0].message
    tool_calls = getattr(response_message, "tool_calls", [])

    if tool_calls:
        call_functions(tool_calls)
    else:
        display_wrapped(response_message.content)

In [ ]:
process_message("What is the revenue for the region of Africa?")

In [ ]:
process_message("What is the revenue for the region of north america?")

In [ ]:
process_message("What is the revenue for all regions?")

In [ ]:
process_message("What were the sales for April 2023?")

In [ ]:
process_message("What is the meaning of life?")